# Nlp final project

1. Yaniv Gabay - 205745615 - yanivga@edu.hac.ac.il
2. Shahar Asher - 209305408 - shaharas@edu.hac.ac.il
3. Hadar Liel Harush - 211721568 - hadarhar@edu.hac.ac.il
## Eminem song generator

This project will compare different models, all trained on the legendary hip-hop artist Eminem lyrics.
We want to expirment with different models, our own model, different apis, to see and compare the results.


to be added?

In [2]:
# Essential imports for handling arrays, dataframes, and neural network functionalities
import os
import torch
import pandas as pd

# Importing models and tokenizers from the Hugging Face Transformers library
from transformers import GPT2Model, GPT2Tokenizer

# Natural Language Processing (NLP) tools
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Downloading necessary NLTK data, consider running these lines only once
nltk.download('wordnet', quiet=True)  # WordNet Lemmatizer
nltk.download('punkt', quiet=True)    # Punkt Tokenizer models

# Our utilities file
import utilities as utils
# Optional: Uncomment the following line for CUDA debugging
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


c:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Removed non-ASCII characters: 
Cleaned String: This is a testwith the character.


In [3]:
##### Loading the data
### we be using <startVerse> <endVerse> <startChorus><endChorus>
## to give our model some different between the verses and the choruses.


##### you can change this to export the data
##### through the process 
debug = False
file_loc = './Eminem_Lyrics.csv'
df = utils.import_data_from_location(file_loc)

utils.export_data_to_csv(df,'afterImport-1',debug)



Error with encoding utf-8: 'utf-8' codec can't decode byte 0x92 in position 6: invalid start byte
Success with encoding: latin1


In [4]:
#actual messing with the dataset
import re


def insert_tokens(lyrics):

    """
    Inserts special tokens at the start and end of each verse and chorus in the lyrics.
    
    Parameters:
        lyrics (str): The lyrics of a song.
        
    Returns:
        str: The processed lyrics with inserted tokens.
    """
    # Pattern to identify all tags (verses, choruses, etc.)
    pattern = re.compile(r'\[(Verse|Chorus|Intro|Outro)')

    # Placeholder for processed lyrics
    processed_lyrics = "<startSong>\n"
    last_tag = None

    for line in lyrics.split('\n'):
        tag_match = pattern.match(line)
        if tag_match:
            # Close the previous tag if exists
            if last_tag:
                processed_lyrics += f"<end{last_tag}>\n"
            # Update last_tag and open a new tag
            last_tag = tag_match.group(1)  # Capture the tag name (Verse or Chorus)
            processed_lyrics += f"<start{last_tag}>\n"
        else:
            processed_lyrics += line + "\n"

    # Close the last opened tag
    if last_tag:
        processed_lyrics += f"<end{last_tag}>\n"
    processed_lyrics += "<endSong>"

    return processed_lyrics




## some songs can have some another tags, like post-chorus,pre etc.
## we arent intresnted in those, so we will just delete them in the next step
new_df = df['Lyrics'].apply(insert_tokens)
new_df = new_df.apply(utils.remove_tags)
new_df = new_df.apply(utils.handle_special_characters)
#lowercase everything
new_df = new_df.str.lower()

utils.export_data_to_csv(new_df,'afterLowerAndRemoveSpecialChars!',True)


In [5]:

clean = new_df.apply(utils.remove_non_ascii_characters)
clean = clean.apply(utils.expand_contractions, args=(utils.contractions_dict,))

utils.export_data_to_csv(clean,'afterRemoveNonAscii',True)



Removed non-ASCII characters: 
Removed non-ASCII characters: á
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: ó
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: é
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: ö
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: âé
Removed non-ASCII characters: ó
Removed non-ASCII characters: 
Removed non-ASCII characters: óá
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: 
Removed non-ASCII characters: óáíé
Removed non-ASCII charac

## we will try several tokenizer options

In [6]:
temp_switch = {'<startverse>':'startverse',
                '<endverse>':'endverse',
                '<startchorus>':'startchorus',
                '<endchorus>':'endchorus',
                '<startoutro>':'startoutro',
                '<endoutro>':'endoutro',
                '<startintro>':'startintro',
                '<endintro>':'endintro',
                '<startsong>':'startsong',
                '<endsong>':'endsong'}
def replace_tags(text, temp_switch, reverse=False):
    if reverse:
        temp_switch = {v: k for k, v in temp_switch.items()}

    for tag, replacement in temp_switch.items():
        text = text.replace(tag, replacement)
    return text

temp_clean = clean.apply(lambda x: replace_tags(x, temp_switch, reverse=False))
utils.export_data_to_csv(temp_clean,'temp_clean',debug)
!pip install spacy
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
spacy_tokens_df = temp_clean.apply(lambda x: [token.text for token in nlp(x)])

utils.export_data_to_csv(spacy_tokens_df,'LATESTSPACETOKENS',True)


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.8 MB 328.2 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.8 MB 554.9 kB/s eta 0:00:23
      --------------------------------------- 0.3/12.8 MB 1.0 MB/s eta 0:00:13
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     --- ------------------------------------ 1.0/12.8 MB 2.8 MB/s eta 0:00:05
     ----- ---------------------------------- 1.9/12.8 MB 4.6 MB/s eta 0:00:03
     ----------- ---------------------------- 3.5/12.8 MB 7.8 MB/s eta 0:00:02
     ----------------- ---------------------- 5.7/12.8 MB 11.4 MB/s eta


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
def build_vocab(series):
    """
    Builds a vocabulary dictionary mapping each unique token to a unique index.
    
    Special tokens are predefined with specific indices. This function iterates through
    each song in the provided series, updating the set of unique tokens. It then assigns
    an index to each token, starting with indices following the special tokens.
    
    Parameters:
        series (pd.Series): A pandas series containing lists of tokenized lyrics of songs.
        
    Returns:
        dict: A dictionary mapping tokens to their respective indices in the vocabulary.
    """
    unique_tokens = set()
    for tokens_list in series:
        unique_tokens.update(tokens_list)
    
    token_to_idx = {
        '<PAD>': 0,
        '<startsong>': 1,
        '<endsong>': 2,
        '<startverse>': 3,
        '<endverse>': 4,
        '<startchorus>': 5,
        '<endchorus>': 6,
        '<startintro>': 7,
        '<endintro>': 8
    }

    # Continue indexing from the number after the special tokens
    for token in unique_tokens:
        if token not in token_to_idx:
            token_to_idx[token] = len(token_to_idx)

    return token_to_idx



def tokens_to_indices(tokenized_song, token_to_idx):
    """
    Converts a list of tokens from a song to their corresponding indices based on
    a given vocabulary mapping.
    
    If a token is not found in the mapping, it is replaced by the index of the '<PAD>'
    token, serving as a placeholder for unknown tokens.
    
    Parameters:
        tokenized_song (list of str): A list containing the tokens of a song.
        token_to_idx (dict): A dictionary mapping tokens to their respective indices.
        
    Returns:
        list of int: A list of indices corresponding to the tokens of the input song.
    """
    # Convert each token to its index, defaulting to the '<PAD>' token's index if not found
    return [token_to_idx.get(token, token_to_idx['<PAD>']) for token in tokenized_song]



In [25]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class TokenSongsDataset(Dataset):
    """
    A PyTorch Dataset class for sequences of tokens from song lyrics.
    
    This class prepares sequences of a specified length from the tokenized lyrics,
    suitable for training models on sequence prediction tasks, such as language modeling.
    
    Parameters:
        series (pd.Series): A pandas series containing lists of tokenized lyrics.
        seq_length (int): The length of the sequences to be prepared for the model.
    """
    def __init__(self, series, seq_length):
        self.series = series
        self.seq_length = seq_length
        self.tokens = self.prepare_tokens()

    def prepare_tokens(self):
        """
        Flattens all token lists from the series into a single list.
        
        Returns:
            list: A list of all tokens from the series combined.
        """
        # Efficient flattening using itertools.chain
        from itertools import chain
        return list(chain.from_iterable(self.series))

    def __getitem__(self, idx):
        """
        Retrieves a sequence of tokens and its corresponding target sequence from the dataset.
        
        The target sequence is the original sequence shifted by one token to the right.
        
        Parameters:
            idx (int): The start index of the sequence.
            
        Returns:
            tuple: A tuple containing the input sequence and target sequence tensors.
        """
        # Ensure the sequence does not exceed the tokens list's bounds
        end_idx = min(idx + self.seq_length, len(self.tokens) - 1)
        
        # Sequence from 'idx' to 'end_idx'
        seq = self.tokens[idx:end_idx]
        # Target sequence is offset by one token
        target_seq = self.tokens[idx + 1:end_idx + 1]

        # Padding for sequences shorter than 'seq_length' at the end of the tokens list
        seq += [0] * (self.seq_length - len(seq))  # Assuming 0 is the index for '<PAD>'
        target_seq += [0] * (self.seq_length - len(target_seq))  # Assuming 0 is the index for '<PAD>'

        # Conversion to tensors
        seq_tensor = torch.tensor(seq, dtype=torch.long)
        target_tensor = torch.tensor(target_seq, dtype=torch.long)

        return seq_tensor, target_tensor

    def __len__(self):
        """
        Returns the adjusted length of the dataset accounting for the sequence length.
        
        This adjustment ensures that each sequence has a corresponding target sequence.
        
        Returns:
            int: The adjusted length of the dataset.
        """
        # Adjust length to ensure there's a target for each input sequence
        return max(0, len(self.tokens) - self.seq_length)


In [29]:

import torch.nn as nn

class BiLSTM(nn.Module):
    """
    A Bidirectional LSTM model for sequence prediction tasks.
    
    Parameters:
        vocab_size (int): The size of the vocabulary.
        embedding_dim (int): The dimensionality of the embedding layer.
        hidden_dim (int): The dimensionality of the LSTM's hidden states.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(p=0.5)  # Adjust the dropout rate as needed
        # Ensure hidden_dim is divisible by 2 for the bidirectional LSTM
        assert hidden_dim % 2 == 0, "Hidden dimension must be even for BiLSTM."
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=2, dropout=0.5, bidirectional=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sequence):
        """
        Defines the forward pass of the model.
        
        Parameters:
            sequence (Tensor): The input sequence to the model.
            
        Returns:
            Tensor: The output predictions for the next token in the sequence.
        """
        embedded = self.embedding(sequence)
        embedded = self.dropout(embedded)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output



def get_vocab_size_from_series(tokenized_songs_series):
    """
    Determines the vocabulary size from a series of tokenized songs.
    
    Parameters:
        tokenized_songs_series (pd.Series): A series of tokenized songs.
        
    Returns:
        int: The size of the vocabulary.
    """
    vocabulary = set().union(*tokenized_songs_series)
    return len(vocabulary)
def replace_tags_in_list(tokens, temp_switch, reverse=False):
    if reverse:
        temp_switch = {v: k for k, v in temp_switch.items()}

    # Replace each token if it matches a tag in temp_switch
    return [temp_switch.get(token, token) for token in tokens]

# Apply the adapted function to the tokenized data
spacy_tokens_df = spacy_tokens_df.apply(lambda tokens_list: replace_tags_in_list(tokens_list, temp_switch, reverse=True))

utils.export_data_to_csv(spacy_tokens_df,'LATESTSPACETOKENS',debug)
# Build vocabulary from tokenized series and convert songs to indices
token_to_idx = build_vocab(spacy_tokens_df)

# Check if special tokens are in the vocabulary
special_tokens = ['<startverse>', '<endverse>', '<startchorus>', '<endchorus>', '<startintro>', '<endintro>', '<startsong>', '<endsong>']
for token in special_tokens:
    assert token in token_to_idx, f"{token} not in vocabulary"

indexed_tokens_series = spacy_tokens_df.apply(lambda song: tokens_to_indices(song, token_to_idx))

# Sample a few tokenized songs to inspect the placement of special tokens
for idx, song in indexed_tokens_series.sample(5).items():
    print(f"Song {idx}: {song}")  # Print the first 50 tokens of each sampled song
    print("...")
    print()


# Define the sequence length and initialize the dataset
seq_length = 70
dataset = TokenSongsDataset(indexed_tokens_series, seq_length)

# Initialize DataLoader with the dataset
batch_size = 12
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the BiLSTM model
embedding_dim = 100
hidden_dim = 256
model = BiLSTM(len(token_to_idx), embedding_dim, hidden_dim)

# Create a reverse mapping from indices to tokens for later use
idx_to_token = {idx: token for token, idx in token_to_idx.items()}


Song 266: [1, 7, 12451, 4459, 9355, 11314, 7448, 6506, 13359, 13359, 1147, 8303, 5845, 9355, 3588, 5845, 8303, 9835, 9355, 7450, 8835, 2488, 325, 12865, 8748, 9355, 7688, 10282, 1056, 325, 12865, 9355, 3588, 5845, 7448, 6506, 7448, 325, 9400, 325, 3588, 9355, 7596, 325, 9400, 325, 7216, 9355, 7448, 6506, 325, 9400, 325, 7216, 9355, 7596, 325, 1685, 9688, 8303, 5243, 9355, 7448, 4750, 7448, 5918, 11169, 4934, 6216, 13399, 325, 7688, 6912, 3752, 7448, 6910, 13248, 4466, 7216, 13370, 9355, 12865, 9355, 3588, 5845, 7448, 6506, 7448, 9355, 12865, 325, 3588, 9355, 7596, 10358, 12867, 5845, 12671, 7448, 325, 1685, 9688, 8303, 873, 9355, 325, 7688, 5243, 9355, 7448, 4750, 7448, 5918, 11169, 4934, 6216, 13399, 3252, 1015, 7448, 9355, 2826, 5845, 2398, 6086, 7448, 9355, 7813, 5845, 10016, 8, 3, 10358, 12867, 5845, 12671, 8636, 11116, 10421, 4934, 10297, 5782, 1855, 325, 1685, 5782, 9360, 4839, 11638, 8343, 9028, 3321, 4002, 9360, 325, 10790, 1592, 13248, 12738, 4162, 8250, 5845, 1524, 4934, 1029

In [ ]:
import torch
import torch.nn as nn

##### we want to have some basic song strcture
##### so we give him a penalty if he doesnt follow the BASIC structure
##### we didnt want to be so strict, so we give him a penalty of 1.0


class SongStructureLoss(nn.Module):
    def __init__(self, token_to_idx, structure_penalty=1.0):
        super(SongStructureLoss, self).__init__()
        self.base_loss = nn.CrossEntropyLoss()
        self.structure_penalty = structure_penalty
        # Store indices for special tokens
        self.start_song_idx = token_to_idx['<startSong>']
        self.end_song_idx = token_to_idx['<endSong>']
        self.start_chorus_idx = token_to_idx['<startChorus>']
        self.end_chorus_idx = token_to_idx['<endChorus>']

    def forward(self, outputs, targets):
        # Calculate the base cross-entropy loss
        base_loss = self.base_loss(outputs, targets)

        # Penalty for missing structure
        structure_penalty = 0.0

        batch_size = targets.size(0)
        for i in range(batch_size):
            # Extract the target sequence for the current batch item
            target_sequence = targets[i]

            # Check for <startSong> at the beginning and <endSong> at the end
            correct_start = (target_sequence[0] == self.start_song_idx)
            correct_end = (target_sequence[-1] == self.end_song_idx)

            # Check for at least one <startChorus> before an <endChorus>
            start_chorus_positions = (target_sequence == self.start_chorus_idx).nonzero(as_tuple=True)[0]
            end_chorus_positions = (target_sequence == self.end_chorus_idx).nonzero(as_tuple=True)[0]

            # Ensure there's at least one start_chorus before one end_chorus
            correct_chorus_sequence = False
            if start_chorus_positions.nelement() > 0 and end_chorus_positions.nelement() > 0:
                correct_chorus_sequence = start_chorus_positions[0] < end_chorus_positions[-1]

            # If any of the structure conditions are not met, apply the penalty
            if not (correct_start and correct_end and correct_chorus_sequence):
                structure_penalty += self.structure_penalty

        # Normalize the structure penalty by batch size
        structure_penalty /= batch_size

        # Combine the base loss with the structure penalty
        total_loss = base_loss + structure_penalty
        return total_loss


In [ ]:
import numpy as np
import torch

song_structure_loss = SongStructureLoss(token_to_idx, structure_penalty=1.0)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transfer the model to the configured device
model.to(device)

# Number of training epochs
num_of_epochs = 7

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Lists to store per-epoch metrics
epoch_losses = []
epoch_accuracies = []
epoch_perplexities = []

# Training loop
for epoch in range(num_of_epochs):
    model.train()  # Set the model to training mode
    batch_losses = []
    batch_accuracies = []
    batch_perplexities = []

    for i, (inputs, targets) in enumerate(dataloader):
        # Transfer data to the configured device
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        outputs_flattened = outputs.view(-1, outputs.size(-1))
        targets_flattened = targets.view(-1)

        # Loss calculation
        loss = song_structure_loss(outputs_flattened, targets_flattened)
        batch_losses.append(loss.item())

        # Calculate batch perplexity
        perplexity = torch.exp(loss).item()
        batch_perplexities.append(perplexity)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Calculate batch accuracy
        predicted_indices = outputs_flattened.max(1)[1]
        correct_predictions = predicted_indices.eq(targets_flattened).sum().item()
        accuracy = correct_predictions / targets_flattened.size(0)
        batch_accuracies.append(accuracy)

        # Log training progress
        if i % 900 == 0:
            print(f"Epoch [{epoch+1}/{num_of_epochs}], Batch [{i}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, Perplexity: {perplexity:.4f}")

            # Select a sample to display from the batch
            sample_index = np.random.choice(len(inputs))
            input_sample = inputs[sample_index].tolist()
            predicted_sample = predicted_indices.view(inputs.size(0), -1)[sample_index].tolist()
            target_sample = targets[sample_index].tolist()

            # Convert indices to tokens (assuming 'idx_to_token' is defined)
            input_tokens = [idx_to_token[idx] for idx in input_sample]
            predicted_tokens = [idx_to_token[idx] for idx in predicted_sample]
            target_tokens = [idx_to_token[idx] for idx in target_sample]

            print(f"Input: {' '.join(input_tokens)}")
            print(f"Predicted: {' '.join(predicted_tokens)}")
            print(f"Target: {' '.join(target_tokens)}")
            print('-' * 50)

    # Calculate and store epoch metrics
    epoch_loss = np.mean(batch_losses)
    epoch_accuracy = np.mean(batch_accuracies)
    epoch_perplexity = np.mean(batch_perplexities)

    epoch_losses.append(epoch_loss)
    epoch_accuracies.append(epoch_accuracy)
    epoch_perplexities.append(epoch_perplexity)

    # Log epoch metrics
    print(f"End of Epoch [{epoch+1}], Average Loss: {epoch_loss:.4f}, Average Accuracy: {epoch_accuracy:.4f}, Average Perplexity: {epoch_perplexity:.4f}")
    print('=' * 100)

    # Step the learning rate scheduler
    scheduler.step()


Using device: cpu
Epoch [1/7], Batch [0], Loss: 9.5447, Accuracy: 0.0000, Perplexity: 13971.1162
Input: a quack , or i am back on that whacky tobacca i m a step away from crack , obie smacked the shit out of me i am so slap happy , man , snap me back to reality i am a walkin art gallery , a science museum elephantitis of the nuts , you d probably die if you see them i pull them out , they hit the
Predicted: heels bowls facility yah licking bouncer bucklin genghis giovannis ribs sink striped uppercut unaccommodating news miri pistachios sampler slug father pike grandkids grapes mannot striped donuts escapist gandhi purp bucklin slurp bouncer niah marijuan unscrew perish gotcha traps luggage heels urn threaten jump d.o.c durag statutory drag countin hallowed storyboard striped tan whom bumper dealers enormous leaders ocds durag opening looney holler colombian gandhi morning pontiac magazine spiralin lindsay formula
Target: quack , or i am back on that whacky tobacca i m a step away from 

KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np

def deterministic_sampling(logits):
    """
    Selects the token with the highest logit value.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        
    Returns:
        int: The index of the token with the highest logit.
    """
    return logits.argmax().item()

def temperature_sampling(logits, temperature=1.0):
    """
    Samples a token index based on adjusted logits using a temperature parameter.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        temperature (float): The temperature to adjust the logits.
        
    Returns:
        int: The index of the sampled token.
    """
    adjusted_logits = logits / temperature
    probabilities = torch.softmax(adjusted_logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def top_k_sampling(logits, top_k=0):
    """
    Samples a token from the top K logits.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        top_k (int): The number of top logits to consider for sampling.
        
    Returns:
        int: The index of the sampled token.
    """
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = -float('Inf')
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def top_p_sampling(logits, top_p=0.0):
    """
    Samples a token from the logits, considering only the top P cumulative probabilities.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        top_p (float): The cumulative probability threshold.
        
    Returns:
        int: The index of the sampled token.
    """
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=0), dim=0)
    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = -float('Inf')
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def generate_song(model, start_sequence, token_to_idx, idx_to_token, seq_length, device, max_length=100, method='none', temperature=1.0, top_k=0, top_p=0.0):
    """
    Generates a song sequence using a specified sampling method.
    
    Parameters:
        model (nn.Module): The trained model for generating sequences.
        start_sequence (list of str): The initial sequence of tokens to start generating from.
        token_to_idx (dict): Mapping from tokens to their indices.
        idx_to_token (dict): Mapping from indices back to tokens.
        seq_length (int): The sequence length that the model expects.
        device (torch.device): The device to perform the computation on.
        max_length (int, optional): The maximum length of the generated sequence. Defaults to 100.
        method (str, optional): The sampling method to use ('none', 'temperature', 'top-k', 'top-p'). Defaults to 'none'.
        temperature (float, optional): Temperature parameter for temperature sampling. Defaults to 1.0.
        top_k (int, optional): Top K parameter for top-k sampling. Defaults to 0.
        top_p (float, optional): Top P parameter for top-p sampling. Defaults to 0.0.
        
    Returns:
        list of str: The generated sequence of tokens.
    """
    model.eval()
    tokens = [token_to_idx.get(token, token_to_idx['<PAD>']) for token in start_sequence]
    sampling_methods = {
        'none': deterministic_sampling,
        'temperature': lambda logits: temperature_sampling(logits, temperature),
        'top-k': lambda logits: top_k_sampling(logits, top_k),
        'top-p': lambda logits: top_p_sampling(logits, top_p),
    }
    sampling_function = sampling_methods[method]

    with torch.no_grad():
        for _ in range(max_length):
            sequence = torch.tensor(tokens[-seq_length:], dtype=torch.long).unsqueeze(0).to(device)
            output = model(sequence)
            last_output = output[0, -1, :]

            predicted_token_idx = sampling_function(last_output)
            tokens.append(predicted_token_idx)

            if predicted_token_idx == token_to_idx.get('<ENDSONG>', token_to_idx['<PAD>']):
                break

    return [idx_to_token.get(idx, '<UNK>') for idx in tokens]

# Example usage
outputs = []
methods = ['none', 'temperature', 'top-k', 'top-p']
start_sequence = ['<startsong>']
for method in methods:
    print(f"Method: {method}")
    generated_song = generate_song(model, start_sequence, token_to_idx, idx_to_token, seq_length, device, max_length=300, method=method, temperature=0.8, top_k=10, top_p=0.8)
    generated_song_text = ' '.join(generated_song)
    outputs.append(generated_song_text)
    print(generated_song_text)
    print('-' * 50)


### we can try to mess more with the outputs